# FanDuel data

The purpose of this file is to take in an upcoming FanDuel competition (csv) and generate the inputs necessary for the ML model

In [10]:
import nbimporter
import pandas as pd
from datetime import datetime
import player_matches_data as pm_data
import games_data
import pickle
import numpy as np

Importing Jupyter notebook from player_matches_data.ipynb
Importing Jupyter notebook from games_data.ipynb


## Create games dataframe (input to model)

In [11]:
def add_home_game(row):
    return row['team_key'] == row['Game'].split("@")[1]

In [12]:
def add_date(data):
    df = data.copy()
    
    date = datetime.today()
    year = date.year
    month = date.month
    day = date.day

    df['date'] = date
    df['year'] = year
    df['month'] = month
    df['day'] = day

    return df

In [13]:
def add_player_id(row, name_id_mapping):
    name = row['Nickname']
    ids = name_id_mapping[name_id_mapping.name == name]['player_id']
    
    if len(ids) > 0:
        return ids.iloc[0]
    else:
        print("Please update player_id for name " + name)
        return None

In [14]:
def add_stats_to_data(data, stats):
    df = data.copy()
    
    for stat in stats:
        df[stat] = 0

    return df

In [15]:
def get_competition_for_fanduel(fanduel):
    competition = fanduel[fanduel['Injury Indicator'] != "O"].copy()
        
    name_id_mapping = pd.read_csv("mappings/name_id_mapping.csv")
    competition['Id'] = competition.apply(lambda x: add_player_id(x, name_id_mapping), axis = 1)
    competition = competition.rename(columns={'Id': 'player_id'})
    competition = competition.dropna(subset=['player_id'])
    competition = competition.drop(columns=['First Name', 'Last Name', 'Injury Details', 'Tier'])
    
    return competition.dropna(how='all', axis=1)

In [16]:
def get_model_input_for_competition(competition, player_matches, player_seasons, rolling_games_window, duplicate=False):
    adj_comp = competition.copy()
    adj_comp['team_key'] = adj_comp['Team']
    adj_comp['opponent_key'] = adj_comp['Opponent']
    adj_comp['home_game'] = adj_comp.apply(add_home_game, axis = 1)
    adj_comp = add_date(adj_comp)

    adj_comp = pm_data.add_game_ids(adj_comp)
    adj_comp = pm_data.add_season_start_year(adj_comp)
    
    stats = ['secs', 'fgm', 'fga', '3pm', '3pa', 'ftm', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tvr', 'pf', 'fp']
    fd_complete = add_stats_to_data(adj_comp, stats)

    fd_complete = pm_data.roll_with_old_data(fd_complete, player_matches, stats, rolling_games_window)

    per_game_df = games_data.create_games_data(fd_complete, player_seasons, 7, ROLLING_GAMES_WINDOW, duplicate)
    
    return per_game_df

### Testing 

In [17]:
player_seasons = pd.read_csv("../data/raw_data/2009_2019_player_seasons_unique.csv")
player_matches = pd.read_csv("../data/raw_data/20100101_20191129_player_matches.csv")
ROLLING_GAMES_WINDOW = 5

In [18]:
competition_key = "2019-12-01-41164"
fanduel = pd.read_csv("../data/competitions_fanduel/FanDuel-NBA-" + competition_key + "-players-list.csv")

### Real competition flow

In [19]:
competition = get_competition_for_fanduel(fanduel)

Please update player_id for name Nigel Williams-Goss
Please update player_id for name Garrison Mathews
Please update player_id for name Justin Robinson
Please update player_id for name Mfiondu Kabengele
Please update player_id for name Alen Smailagic
Please update player_id for name Oshae Brissett
Please update player_id for name Juwan Morgan
Please update player_id for name Shamorie Ponds
Please update player_id for name Amir Coffey
Please update player_id for name BJ Johnson
Please update player_id for name Dewan Hernandez
Please update player_id for name Jarrell Brantley
Please update player_id for name Melvin Frazier
Please update player_id for name Miye Oni
Please update player_id for name Justin Wright-Foreman


In [20]:
model_input = get_model_input_for_competition(competition, player_matches, player_seasons, ROLLING_GAMES_WINDOW)

In [12]:
model_input.to_csv("../data/model_inputs/" + competition_key + ".csv", index = False)
competition.to_csv("../data/competitions_clean/" + competition_key + ".csv", index = False)

### Fake competition flow

In [21]:
keys = [
    "2011-01-14",
    "2011-02-10",
    "2011-03-05",
    "2011-11-17",
    "2011-12-05",
    "2012-01-14",
    "2012-02-10",
    "2012-03-01",
    "2012-03-05",
    "2012-11-17",
    "2012-12-05",
    "2013-01-14",
    "2013-02-10",
    "2013-03-05",
    "2013-10-30",
    "2013-11-17",
    "2014-01-11",
    "2014-01-14",
    "2014-02-10",
    "2014-02-11",
    "2014-03-05",
    "2014-03-12",
    "2014-11-17",
    "2014-12-05",
    "2015-01-11",
    "2015-01-14",
    "2015-02-10",
    "2015-02-24",
    "2015-03-05",
    "2015-03-13",
    "2015-04-11",
    "2015-11-27",
    "2015-12-05",
    "2016-01-11",
    "2016-01-14",
    "2016-02-10",
    "2016-02-24",
    "2016-03-05",
    "2016-03-13",
    "2016-04-11",
    "2016-11-27",
    "2016-12-05",
    "2017-01-11",
    "2017-01-14",
    "2017-02-10",
    "2017-02-24",
    "2017-03-05",
    "2017-03-13",
    "2017-04-11",
    "2017-11-27",
    "2017-12-05",
]

In [24]:
for key in keys:
    try:
        competition = pd.read_csv("../data/competitions_clean/" + key + "_fake.csv")
        model_input = get_model_input_for_competition(competition, player_matches, player_seasons, ROLLING_GAMES_WINDOW, True)
        model_input.to_csv("../data/model_inputs/" + key + "_fake_dup.csv", index = False)
    except Exception as e:
        print("exception: " + str(e))
        continue

exception: [Errno 2] File b'../data/competitions_clean/2011-11-17_fake.csv' does not exist: b'../data/competitions_clean/2011-11-17_fake.csv'
exception: [Errno 2] File b'../data/competitions_clean/2011-12-05_fake.csv' does not exist: b'../data/competitions_clean/2011-12-05_fake.csv'
